
```
███╗   ███╗██╗███╗   ██╗███████╗ ██████╗ ██████╗ ██╗      █████╗ ██████╗ 
████╗ ████║██║████╗  ██║██╔════╝██╔════╝██╔═══██╗██║     ██╔══██╗██╔══██╗
██╔████╔██║██║██╔██╗ ██║█████╗  ██║     ██║   ██║██║     ███████║██████╔╝
██║╚██╔╝██║██║██║╚██╗██║██╔══╝  ██║     ██║   ██║██║     ██╔══██║██╔══██╗
██║ ╚═╝ ██║██║██║ ╚████║███████╗╚██████╗╚██████╔╝███████╗██║  ██║██████╔╝
╚═╝     ╚═╝╚═╝╚═╝  ╚═══╝╚══════╝ ╚═════╝ ╚═════╝ ╚══════╝╚═╝  ╚═╝╚═════╝ 
```
**Run a Minecraft Server on Google Colab!**

---

The script below will run your server. You'll have to create a server first to be able to use it - don't worry, the scripts below will do the majority of the work for you. You might also want to change the default region to your region, check below.

In [ ]:
import os
import re
import json

# Update the package lists
!sudo apt update &>/dev/null && echo "apt cache successfully updated" || echo "apt cache update failed, you might receive stale packages"
# Install OpenJDK 17
# !wget -qO - https://adoptopenjdk.jfrog.io/adoptopenjdk/api/gpg/key/public | sudo apt-key add -
# !sudo add-apt-repository --yes https://adoptopenjdk.jfrog.io/adoptopenjdk/deb/ &>/dev/null || echo "Failed to add repo. Still can be ignored if openjdk17 gets installed."
!sudo apt-get install openjdk-17-jre-headless &>/dev/null && echo "Yay! Openjdk17 has been successfully installed." || echo "Failed to install OpenJdk17."
#Perform java version check
java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'
if java_ver[0] == "17" :
  print("Openjdk17 is working correctly, you are good to go.")
else:
  print("Openjdk17 doesn't seems to be installed or isn't working, falling back to java", java_ver[0], ". You might experience reduced performance. Minecraft 1.17 and above might fail to launch.")

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Change directory to the Minecraft server folder on Google Drive
%cd "/content/drive/My Drive/Minecraft-server"
!ls #list the directory contents (to verify that working directory was changed)

# Import config file.
if os.path.isfile("colabconfig.json"):
  colabconfig = json.load(open("colabconfig.json"))
else:
  colabconfig = {"server_type": "generic"} # using default, if config doesn't exists.
  json.dump(colabconfig, open("colabconfig.json",'w'))

# Server jar names.
jar_list = {'paper': 'server.jar', 'fabric': 'fabric-server-launch.jar', 'generic': 'server.jar'}
jar_name = jar_list[colabconfig["server_type"]]

# Java arguments.
if colabconfig["server_type"] == "paper":
  server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"
else:
  server_flags = "" # aiker's flags might negatively impact performance on non-paper servers.
memory_allocation = "-Xmx6144M -Xms6144M"

# Chose the tunnle service you want to use
# Available options: ngrok, argo, playit
tunnel_service = "argo"
print("Procedding to use", tunnel_service)

if tunnel_service == "ngrok":
  !pip -q install pyngrok
  from pyngrok import conf, ngrok
  
  # Ask for the ngrok authtoken
  print("Get your authtoken from https://dashboard.ngrok.com/auth")
  import getpass
  authtoken = getpass.getpass()  # input your Ngrok auth token everytime you run the cell or simply replace "getpass.getpass()" with your token in "double quotes"
  ! ngrok authtoken $authtoken # login to ngrok

  # Sets default ngrok region
  conf.get_default().region = 'us'  # Change this to whichever region you want

  # Connect to ngrok
  url = ngrok.connect(25565, 'tcp')
  print('Your server address is ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', ''))

  print('Starting server...')
  !java $memory_allocation $server_flags -jar $jar_name nogui

elif tunnel_service == "playit":
  ! curl -SsL https://playit-cloud.github.io/ppa/key.gpg | sudo apt-key add -
  ! sudo curl -SsL -o /etc/apt/sources.list.d/playit-cloud.list https://playit-cloud.github.io/ppa/playit-cloud.list
  ! sudo apt update &>/dev/null && sudo apt install playit &>/dev/null && echo "Playit.gg installed" || echo "Failed to install playit"
  print('Starting server...')
  ! playit & java $memory_allocation $server_flags -jar $jar_name nogui

else:
  # Download & make argo executable
  !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
  !chmod +x cloudflared-linux-amd64
  print('Starting server...')
  !./cloudflared-linux-amd64 tunnel --url tcp://127.0.0.1:25565 & java $memory_allocation $server_flags -jar $jar_name nogui


apt cache successfully updated
Yay! Openjdk17 has been successfully installed.
Openjdk17 is working correctly, you are good to go.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Minecraft-server
banned-ips.json		   commands.yml  permissions.yml       versions
banned-players.json	   config	 plugins	       whitelist.json
bukkit.yml		   eula.txt	 server.jar	       world
cache			   help.yml	 server.properties     world_nether
cloudflared-linux-amd64    libraries	 spigot.yml	       world_the_end
cloudflared-linux-amd64.1  logs		 usercache.json
colabconfig.json	   ops.json	 version_history.json
Procedding to use argo
--2023-04-15 15:02:13--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response...

# Make-a-Server

The code below will download a server for you and accept the EULA. After running these scripts, your server will be ready to run.

**Download the Minecraft server**

The code below will download Paper, a high-performance fork of the Vanilla server.
Other server platforms can be used by placing the server.jar in the Drive folder manually.

In [1]:
# Replace "1.19.2" with your desired server version.
# Available versions:
# - 1.19.2
# - 1.19.1
# - 1.19
# - 1.18.2
# - 1.18.1
# - 1.18
# - 1.17.1
# - 1.17
# - 1.16.4
# - 1.16.3
# - 1.15.2
# - 1.14.4
# - 1.13.2
# - 1.12.2
# - 1.11.2
# - 1.10.2
# - 1.9.4
# - 1.8.8
# Newer versions might work too, however this isn't guaranteed.
version = '1.19.3'

#Chose server type. Currently available versions: fabric, paper
server_type = 'paper'

# Print experimental build warning -- no longer needed
# if version == '1.18' and server_type == 'paper':
#  print("<!> Paper 1.18 builds are still experimental. Make regular Backups!")

from google.colab import drive
import requests
import json


drive.mount('/content/drive')

# Create the directory which will be used for the server
! mkdir "/content/drive/My Drive/Minecraft-server"
%cd "/content/drive/My Drive/Minecraft-server"

# Internal init...


a = requests.get("https://papermc.io/api/v2/projects/paper/versions/" + version)
#print(a.json()["builds"][-1])
b = requests.get("https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]))
#print(b.json()["downloads"]["application"]["name"])
print("https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]) + "/downloads/" + b.json()["downloads"]["application"]["name"])

paperURL = "https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]) + "/downloads/" + b.json()["downloads"]["application"]["name"]

jar_name = {'paper': 'server.jar', 'fabric': 'fabric-installer.jar'}
url = {
    'paper': (paperURL),
    'fabric': 'https://maven.fabricmc.net/net/fabricmc/fabric-installer/0.7.4/fabric-installer-0.7.4.jar'
    }

print('Downloading to Google Drive...')

r = requests.get(url[server_type])

if r.status_code is 200:
  with open('/content/drive/My Drive/Minecraft-server/' + jar_name[server_type], 'wb') as f:
    f.write(r.content)
else:
  print('Error '+ str(r.status_code) + '! Most likely you entered a unsupported version. Try running the code again if you think that shouldn\'t have happened.')

# Running specific install path.
if server_type == 'fabric':
  !java -jar fabric-installer.jar server -mcversion $version -downloadMinecraft

# Saving config
colabconfig = {"server_type": server_type}
json.dump(colabconfig, open("colabconfig.json",'w'))

print('Done!') # todo: Would show even after erroring.


<>:63: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:63: SyntaxWarning: "is" with a literal. Did you mean "=="?


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/content/drive/My Drive/Minecraft-server’: File exists
/content/drive/My Drive/Minecraft-server
https://papermc.io/api/v2/projects/paper/versions/1.19.3/builds/448/downloads/paper-1.19.3-448.jar
Done!


<ipython-input-1-3c600eb5057a>:63: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if r.status_code is 200:


**Automatically accept the EULA**

In [2]:
# Please read the file stored in your server folder before running this command. 
# Also, go to https://www.minecraft.net/en-us/eula to read Minecraft's EULA.

# Make sure Drive is mounted
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/My Drive/Minecraft-server"
!echo "eula=true" >> eula.txt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Minecraft-server


# Debug
SSH access to host OS - Thanks to [colab-ssh](https://github.com/WassimBenzarti/colab-ssh).

In [ ]:
#@title Colab-ssh tunnel
#@markdown Execute this cell to open the ssh tunnel. Check [colab-ssh documentation](https://github.com/WassimBenzarti/colab-ssh) for more details.

# Install colab_ssh on google colab
!pip install colab_ssh --upgrade

from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
ssh_tunnel_password = "<PUT_YOUR_PASSWORD_HERE>" #@param {type: "string"}
launch_ssh_cloudflared(password=ssh_tunnel_password)

In [ ]:
#Get public address (ngrok)
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [3]:
## For inspecting the minecraft server directory ##
%cd "/content/drive/My Drive/Minecraft-server"
!ls


/content/drive/My Drive/Minecraft-server
banned-ips.json		   commands.yml  permissions.yml       versions
banned-players.json	   config	 plugins	       whitelist.json
bukkit.yml		   eula.txt	 server.jar	       world
cache			   help.yml	 server.properties     world_nether
cloudflared-linux-amd64    libraries	 spigot.yml	       world_the_end
cloudflared-linux-amd64.1  logs		 usercache.json
colabconfig.json	   ops.json	 version_history.json
